In [1]:
using BenchmarkTools

In [2]:
do_stuff1(x) = (x .* 2) .^ 2
do_stuff2(x) = [i < 1_000 ? i*2 : i*3 for i in x]

do_stuff2 (generic function with 1 method)

In [3]:
do_stuff1(1:100_000)

100000-element Vector{Int64}:
           4
          16
          36
          64
         100
         144
         196
         256
         324
         400
         484
         576
         676
           ⋮
 39991200484
 39992000400
 39992800324
 39993600256
 39994400196
 39995200144
 39996000100
 39996800064
 39997600036
 39998400016
 39999200004
 40000000000

In [4]:
do_stuff2(1:100_000)

100000-element Vector{Int64}:
      2
      4
      6
      8
     10
     12
     14
     16
     18
     20
     22
     24
     26
      ⋮
 299967
 299970
 299973
 299976
 299979
 299982
 299985
 299988
 299991
 299994
 299997
 300000

In [10]:
@btime 1:100 .|> x->do_stuff1(1:(10_000+x));

  1.036 ms (203 allocations: 7.68 MiB)


In [11]:
@btime 1:100 .|> x->do_stuff1(1:(100_000+x));

  18.939 ms (203 allocations: 76.34 MiB)


In [12]:
@btime 1:100 .|> x->do_stuff2(1:(10_000+x));

  1.371 ms (201 allocations: 7.68 MiB)


In [13]:
@btime 1:100 .|> x->do_stuff2(1:(100_000+x));

  10.447 ms (201 allocations: 76.34 MiB)


In [9]:
@code_native do_stuff1(1:100_000)

	.text
; ┌ @ In[2]:1 within `do_stuff1`
	pushq	%rbp
	movq	%rsp, %rbp
	pushq	%r15
	pushq	%r14
	pushq	%r12
	pushq	%rsi
	pushq	%rdi
	pushq	%rbx
	subq	$176, %rsp
	vmovdqa	%xmm10, -64(%rbp)
	vmovdqa	%xmm9, -80(%rbp)
	vmovdqa	%xmm8, -96(%rbp)
	vmovdqa	%xmm7, -112(%rbp)
	vmovdqa	%xmm6, -128(%rbp)
	vpxor	%xmm0, %xmm0, %xmm0
	movl	$jl_get_pgcstack, %eax
	movq	%rcx, %rdi
	movq	$0, -144(%rbp)
	vmovdqa	%xmm0, -160(%rbp)
	callq	*%rax
	movq	%rax, %r14
; │┌ @ broadcast.jl:1303 within `broadcasted` @ broadcast.jl:1126
; ││┌ @ range.jl:770 within `first`
; │││┌ @ Base.jl:42 within `getproperty`
	movq	$4, -160(%rbp)
	leaq	-160(%rbp), %rcx
	movq	(%r14), %rax
	movq	%rax, -152(%rbp)
	movq	%rcx, (%r14)
; ││└└
; ││┌ @ int.jl:88 within `*`
	movq	(%rdi), %r12
; ││└
; ││┌ @ range.jl:699 within `length`
; │││┌ @ int.jl:86 within `-`
	movq	8(%rdi), %rsi
	subq	%r12, %rsi
; ││└└
; ││┌ @ range.jl:451 within `StepRangeLen` @ range.jl:451 @ range.jl:441
; │││┌ @ operators.jl:425 within `>=`
; ││││┌ @ int.jl:477 within

# AD in Zygote

$$
{\displaystyle {\tfrac {1}{2}}\mathbf {x} ^{\mathrm {T} }Q\mathbf {x} +\mathbf {c} ^{\mathrm {T} }\mathbf {x} }
$$

In [14]:
using Zygote

In [82]:
x = [2.,1.];

In [83]:
c = [1., 2.];

In [84]:
Q = [2. 3.; 4. 5.];

In [85]:
loss(x) = 0.5*x'*Q*x + c'*x;

In [86]:
loss(x)

17.5

In [87]:
gradient(loss, x)

([8.5, 14.0],)

In [92]:
@code_native loss(x)

	.text
; ┌ @ In[85]:1 within `loss`
	pushq	%rbp
	movq	%rsp, %rbp
	pushq	%r15
	pushq	%r14
	pushq	%r12
	pushq	%rsi
	pushq	%rdi
	pushq	%rbx
	andq	$-32, %rsp
	subq	$128, %rsp
	vxorps	%xmm0, %xmm0, %xmm0
	movl	$jl_get_pgcstack, %eax
	movq	%rdx, %rdi
	vmovaps	%ymm0, 32(%rsp)
	movq	%rdi, 120(%rsp)
	vzeroupper
	callq	*%rax
	movq	%rax, %rsi
	movq	$8, 32(%rsp)
	leaq	32(%rsp), %rcx
	movl	$3810284200, %r12d              # imm = 0xE31C52A8
	movl	$1392, %edx                     # imm = 0x570
	movl	$16, %r8d
	movq	(%rsi), %rax
	movq	%rax, 40(%rsp)
	movq	%rcx, (%rsi)
	movabsq	$jl_gc_pool_alloc, %rax
	movq	(%r12), %rbx
	movq	16(%rsi), %rcx
	movq	(%rdi), %rdi
	movq	%rbx, 48(%rsp)
	callq	*%rax
	leaq	uv_tcp_non_ifs_lsp_ipv4(%r12), %rcx
	leaq	88(%rsp), %r14
	movabsq	$jl_apply_generic, %r15
	movl	$4, %r8d
	movq	%rcx, -8(%rax)
	movl	$3958726016, %ecx               # imm = 0xEBF55D80
	movq	%r14, %rdx
	movq	%rdi, (%rax)
	movq	%rax, 56(%rsp)
	movq	%rcx, 88(%rsp)
	movl	$354907536, %ecx                # imm = 0x1

In [90]:
@code_native gradient(loss, x)

	.text
; ┌ @ interface.jl:74 within `gradient`
	pushq	%rbp
	movq	%rsp, %rbp
	pushq	%r15
	pushq	%r14
	pushq	%r12
	pushq	%rsi
	pushq	%rdi
	pushq	%rbx
	andq	$-32, %rsp
	subq	$128, %rsp
	vxorps	%xmm0, %xmm0, %xmm0
	movl	$jl_get_pgcstack, %eax
	movq	%rdx, %rbx
	movl	%r8d, %edi
	vmovups	%ymm0, 80(%rsp)
	vmovaps	%ymm0, 64(%rsp)
	movq	%rbx, 120(%rsp)
	vzeroupper
	callq	*%rax
	movq	%rax, %r15
	movq	$16, 64(%rsp)
	leaq	64(%rsp), %rdx
	addq	$8, %rbx
; │ @ interface.jl:75 within `gradient`
	leal	-1(%rdi), %r8d
	xorl	%ecx, %ecx
	movq	(%r15), %rax
	movq	%rax, 72(%rsp)
	movq	%rdx, (%r15)
	movabsq	$jl_f_tuple, %rax
	movq	%rbx, %rdx
	callq	*%rax
	decl	%edi
	je	L609
; │┌ @ interface.jl:40 within `pullback`
; ││┌ @ interface.jl:34 within `_pullback`
; │││┌ @ interface.jl:11 within `Context` @ interface.jl:8
	movq	16(%r15), %rcx
; │└└└
	movq	(%rbx), %rsi
	movq	%rax, %r14
; │┌ @ interface.jl:40 within `pullback`
; ││┌ @ interface.jl:34 within `_pullback`
; │││┌ @ interface.jl:11 within `Context` @ interfac